<a href="https://colab.research.google.com/github/sagorbrur/bangla-bert/blob/master/notebook/bangla_bert_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# !/usr/bin/python3.9 -m pip install gensim

In [24]:
import pathlib
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import csv
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import pickle
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import preprocessing
import gensim as gensim
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV


In [1]:
import pandas as pd
dataset_url = 'https://raw.githubusercontent.com/supreme-lab/gender-identification/main/dataset.csv'

df = pd.read_csv(dataset_url)
df = df.dropna(subset=['text', 'gender'])
df.head()

,text,gender
0,ল্যাবে গ্রুপমেটের গ্রাফ নিজে একে দিয়েছি অনেক ...,M
1,"এরকম এক রমজান মাসের স্নিগ্ধ বিকেলে, জনৈক বুয়েট...",M
2,শিবিরবিহীন বুয়েট থেকে শিবিরের কেন্দ্রীয় কমিটির...,M
3,জনৈক বন্ধু তার বান্ধবীর সাথে দেখা করার আগে নিজ...,M
4,খুব ক্লোজ জুনিয়রের সাথে শেরে বাংলা হলের সামনের...,M


In [2]:
df.index.size

15367

In [3]:
def get_male_female_keywords(posts, genders):
  # compute keyword for male and female samples
  male_keywords = []
  female_keywords = []
  for idx, post in enumerate(posts):
    splitted = post.split()
    if genders[idx] == "M":
      male_keywords.extend(splitted)
    else:
      female_keywords.extend(splitted)
  return male_keywords, female_keywords

In [4]:
def get_frequency_from_list(keywords):
  freq = {}
  for word in keywords:
    if word in freq:
      freq[word] = freq[word] + 1
    else:
      freq[word] = 1
  return freq

In [5]:
import numpy as np
import pathlib
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.linear_model import SGDClassifier
import random
from sklearn.feature_selection import SelectKBest, chi2

class DataPreProcessor(object):
  def __init__(self, dataset_url):
    self.dataset_url = dataset_url

  def preprocess(self):
    # stop word list
    stop_word_file = 'https://raw.githubusercontent.com/supreme-lab/gender-identification/main/stopwords-bn.txt'
    sw_df = pd.read_csv(stop_word_file, delimiter="\t", header=None)
    stopword_list = sw_df.values.tolist()

    # load the training data
    df = pd.read_csv(self.dataset_url)

    # preprocess step 0: remove duplicate rows
    df = df.drop_duplicates()
    print("after removing duplicate rows, row count: ", df.index.size)

    # preprocess step 1: remove null rows
    df = df.dropna(subset=['text', 'gender'])
    print("after removing null rows, row count: ", df.index.size)

    # preprocess step 2: remove non Bangla characters
    author_posts = []
    author_genders = []
    for index, row in df.iterrows():
        tokens = [re.sub(r'[^\u0980-\u09FF ]+', ' ', str(row['text']))]
        tokens_joined = " ".join(tokens)
        if len(tokens_joined) == 0:
          print("tokens_joined.len==0")
        author_posts.append(tokens_joined)
        author_genders.append(row['gender'])

    stopword_list_new = []
    for word in stopword_list:
      stopword_list_new.append(word[0])
    stopword_list = stopword_list_new
    # preprocess step 3: remove stop words
    author_posts_without_stopword = []
    author_genders_without_stopword = []
    i = 0
    for idx, post in enumerate(author_posts):
        i = i + 1
        # if i< 100:
        #   print("before stopword removal sent.len: ", len(post.split()))
        tokens_without_stopword = [word for word in post.split() if not word in stopword_list]
        # if i<100:
        #   print("after stopword removal sent.len: ", len(tokens_without_stopword))
        tokens_without_stopword_joined = " ".join(tokens_without_stopword)
        if len(tokens_without_stopword_joined) > 0:
          # print("tokens_without_stopword_joined: ", tokens_without_stopword_joined)
          # print("len(tokens_without_stopword_joined) == 0")
          author_posts_without_stopword.append(tokens_without_stopword_joined)
          author_genders_without_stopword.append(author_genders[idx])
    
    post_count = len(author_posts_without_stopword)
    gender_count = len(author_genders_without_stopword)

    print(f"after stopword removing posts.len: {post_count}, gender.count: {gender_count}")
    print("author_genders_without_stopword[0:10]")
    print(author_genders_without_stopword[0:10])

    malekeywords, femalekeywords = get_male_female_keywords(author_posts_without_stopword, author_genders)
    
    # compute keyword occurrence in male and female data
    # print("compute keyword occurrence in male and female data")
    male_freq = get_frequency_from_list(malekeywords)
    female_freq = get_frequency_from_list(femalekeywords)

    # filter out keywords with occurrence <10
    # print("filter out keywords with occurrence <10 ")
    features = []
    for key, value in male_freq.items():
      if value >= 10:
        features.append(key)

    for key, value in female_freq.items():
      if value >= 10:
        features.append(key)

    # get the unique keywords
    # print("get the unique keywords")
    featureset = set(features)
    # print("featureset.len: ", len(featureset))

    # keep words which are in the featureset
    # print("keep words which are in the featureset")
    i = 0
    author_posts_with_keywords_in_features = []
    author_genders_with_keywords_in_features = []
    for idx, post in enumerate(author_posts_without_stopword):
        # if i < 100:
        #   print("post.len: ", len(post.split()))
        tokens_filtered_with_features = [word for word in post.split() if word in featureset]
        tokens_filtered_with_features_joined = " ".join(tokens_filtered_with_features)
        # if i<100:
        #   print("tokens_without_keywords_with_occurrence1.len: ", len(tokens_without_keywords_with_occurrence1))
        
        if len(tokens_filtered_with_features_joined) > 0:
          # print("len(tokens_filtered_with_features_joined) == 0")
          author_posts_with_keywords_in_features.append(tokens_filtered_with_features_joined)
          author_genders_with_keywords_in_features.append(author_genders_without_stopword[idx])
    
    post_count = len(author_posts_with_keywords_in_features)
    gender_count = len(author_genders_with_keywords_in_features)

    print(f"after removing out of features posts.len: {post_count}, gender.count: {gender_count}")
    print("author_genders_with_keywords_in_features[0:10]: ", author_genders_with_keywords_in_features[0:10])

    return author_posts_with_keywords_in_features, author_genders_with_keywords_in_features

In [6]:
dataset_url = "https://raw.githubusercontent.com/supreme-lab/gender-identification/main/dataset.csv"
preprocessor = DataPreProcessor(dataset_url)
posts, genders = preprocessor.preprocess()

after removing duplicate rows, row count:  12726
after removing null rows, row count:  12726
after stopword removing posts.len: 10544, gender.count: 10544
author_genders_without_stopword[0:10]
['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F']
after removing out of features posts.len: 10397, gender.count: 10397
author_genders_with_keywords_in_features[0:10]:  ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F']


In [7]:
len(posts), len(genders)

(10397, 10397)

### Deep Learning Models with TFIDF

In [14]:
class ModelDLWithTFIDF():
  def __init__(self, posts, genders, model, batch, epochs) -> None:
    self.batch = batch
    self.epochs = epochs
    self.author_posts = posts

    self.author_genders = genders
    self.model=Sequential()
    if model == 'LSTM':
      self.model.add(LSTM(300, return_sequences=False))
    elif model == 'GRU':
      self.model.add(GRU(300, return_sequences=False))

  def train(self):
    labels = np.asarray(pd.get_dummies(self.author_genders))
    train_x, test_x, train_y, test_y = model_selection.train_test_split(self.author_posts, labels, test_size=0.3)

    male_list = []
    female_list = []

    self.tfidf_vectorizer = TfidfVectorizer(max_features=600, analyzer='word', ngram_range=(1, 3))
    self.tfidf_vectorizer.fit(self.author_posts)

    train_x_tfidf = self.tfidf_vectorizer.transform(train_x)
    test_x_tfidf = self.tfidf_vectorizer.transform(test_x)
    train_x_tfidf_dense = train_x_tfidf.toarray()
    test_x_tfidf_dense = test_x_tfidf.toarray()

    train_x_tfidf_dense = train_x_tfidf_dense.reshape(train_x_tfidf_dense.shape[0],1,train_x_tfidf_dense.shape[-1])
    test_x_tfidf_dense = test_x_tfidf_dense.reshape(test_x_tfidf_dense.shape[0],1,test_x_tfidf_dense.shape[-1])

    # compile model
    self.model.add(Dense(train_y.shape[1], activation='sigmoid'))
    self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    history_model = self.model.fit(train_x_tfidf_dense, train_y, self.batch, self.epochs, validation_split=0.25)
    scores = self.model.evaluate(test_x_tfidf_dense, test_y, verbose=0)
    author_gender_test_predictions = np.argmax(self.model.predict(test_x_tfidf_dense), axis=1)

    author_gender_original_label = np.argmax(test_y, axis=1)
    accuracy = accuracy_score(author_gender_original_label, author_gender_test_predictions)
    precision = precision_score(author_gender_original_label, author_gender_test_predictions)
    recall = recall_score(author_gender_original_label, author_gender_test_predictions)
    f1 = f1_score(author_gender_original_label, author_gender_test_predictions)
    return accuracy, precision, recall, f1

### Deep Learning Models with Word Embedding

In [20]:
class ModelDLWithWordEmbedding(object):
    def __init__(self, cbow=1, cbow_trainable=1, sg=0, sg_trainable=0, data_split=2, batch=32, epochs=5, model_name="GRU"):
        self.model = Sequential()
        self.cbow = cbow
        self.cbow_trainable = cbow_trainable
        self.sg = sg
        self.sg_trainable = sg_trainable
        self.data_split = data_split
        self.batch = batch
        self.epochs = epochs
        self.model_name = model_name


    def get_stop_words(self):
      stop_word_file = 'https://raw.githubusercontent.com/supreme-lab/gender-identification/main/stopwords-bn.txt'
      sw_df = pd.read_csv(stop_word_file, delimiter="\t", header=None)
      return sw_df.values.tolist()

    def convert_text_to_word_vector(self, author_post, author_gender):
      maxlen = 100
      max_words = 1000

      tokenizer = Tokenizer(num_words=max_words)
      tokenizer.fit_on_texts(author_post)
      sequences = tokenizer.texts_to_sequences(author_post)

      word_index = tokenizer.word_index
      data = pad_sequences(sequences, maxlen=maxlen)
      labels = np.asarray(pd.get_dummies(author_gender))
      return data, labels

    def make_dataset(self, X_data, y_data, n_splits):
      X_train_dataset= []
      X_test_dataset = []
      y_train_dataset=  []
      y_test_dataset = []

      for train_index, test_index in KFold(n_splits).split(X_data):
              X_train, X_test = X_data[train_index], X_data[test_index]
              y_train, y_test = y_data[train_index], y_data[test_index]
              X_train_dataset.append(X_train)
              X_test_dataset.append(X_test)
              y_train_dataset.append(y_train)
              y_test_dataset.append(y_test)

      return X_train_dataset, X_test_dataset, y_train_dataset, y_test_dataset

    def get_embedding_vector(self, author_post):
      tokenized_lines = []
      for single_line in author_post:
        tokenized_lines.append(single_line.split())

      if self.cbow == 1:
        # Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
        self.word_model = gensim.models.Word2Vec(tokenized_lines, sg=0, vector_size=300, window=5, min_count=1, workers=4)
      elif self.sg == 1:
        self.word_model = gensim.models.Word2Vec(tokenized_lines, sg=1, vector_size=300, window=5, min_count=1, workers=4)

      embedding_matrix = np.zeros((len(self.word_model.wv.vectors) + 1, 300))  # 29534*300 --> Total word = 29534 ---> prottek word er vector e 300ta element
      for i, vec in enumerate(self.word_model.wv.vectors):
        embedding_matrix[i] = vec
      return embedding_matrix

    def get_model(self, embedding_matrix, author_post_train, author_gender_train):
        model = Sequential()  # 1 * 100 ---
        if self.cbow_trainable == 1:  # 100 * 300  #input_length = 100
            model.add(
                Embedding(len(self.word_model.wv.vectors) + 1, 300, input_length=author_post_train.shape[1], weights=[embedding_matrix],
                          trainable=True))  # size of embedding matrix is 100*300
        elif self.sg_trainable == 1:
            model.add(
                Embedding(len(self.word_model.wv.vectors) + 1, 300, input_length=author_post_train.shape[1], weights=[embedding_matrix],
                          trainable=True))
        else:
            model.add(
                Embedding(len(self.word_model.wv.vectors) + 1, 300, input_length=author_post_train.shape[1], weights=[embedding_matrix],
                          trainable=False))

        # compile model
        if self.model_name == "GRU":
          model.add(GRU(300, return_sequences=False))
        elif self.model_name == "LSTM":
          model.add(LSTM(300, return_sequences=False))

        model.add(Dense(author_gender_train.shape[1], activation="softmax"))
        model.summary()
        model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=['acc'])
        return model

    def train(self, posts, genders):
        author_post, author_gender = posts, genders

        data , labels = self.convert_text_to_word_vector(author_post, author_gender) #size is post_number*100
        X_train_dataset, X_test_dataset, y_train_dataset, y_test_dataset = self.make_dataset(data, labels, self.data_split)
        embedding_matrix = self.get_embedding_vector(author_post) # 29534 * 100 dimension er ekta matrix

        acc_per_fold = []
        loss_per_fold = []
        precision_per_fold = []
        recall_per_fold = []
        f1_score_per_fold = []

        for i in range(len(X_train_dataset)):
            author_post_train = X_train_dataset[i] # 400*100
            author_gender_train = y_train_dataset[i]
            author_post_test = X_test_dataset[i]
            author_gender_test = y_test_dataset[i]
            #batch, epochs = 50, 100
            batch, epochs = self.batch, self.epochs
            model = self.get_model(embedding_matrix, author_post_train, author_gender_train)
            #csv_logger,tensorboard_callback = self.get_csvlogger_tensorflow_callback()
            history_model = model.fit(author_post_train, author_gender_train, batch, epochs,
                                       validation_split=0.2)
            #, callbacks=[csv_logger, tensorboard_callback])
            scores = model.evaluate(author_post_test, author_gender_test, verbose=0)
            # acc_per_fold.append(scores[1] * 100)
            loss_per_fold.append(scores[0])

            author_gender_test_predictions = model.predict(author_post_test)
            author_gender_predicted_label = np.argmax(author_gender_test_predictions, axis=1)
            author_gender_original_label = np.argmax(author_gender_test, axis=1)

            TP = tf.math.count_nonzero(author_gender_predicted_label * author_gender_original_label, dtype=tf.float32)
            TN = tf.math.count_nonzero((author_gender_predicted_label - 1) * (author_gender_original_label - 1),dtype=tf.float32)
            FP = tf.math.count_nonzero(author_gender_predicted_label * (author_gender_original_label - 1), dtype=tf.float32)
            FN = tf.math.count_nonzero((author_gender_predicted_label - 1) * author_gender_original_label, dtype=tf.float32)

            # print("True positive: ", TP, "True negative: ", TN, "False positive: ", FP, "False negative: ", FN)
            print("TP, TN, FP, FN")
            print(TP, TN, FP, FN)

            accuracy = ( TP + TN ) / (TP + FP +FP +FN)
            precision = TP / (TP + FP)
            recall = TP / (TP + FN)
            f1 = 2 * precision * recall / (precision + recall)

            accuracy = accuracy_score(author_gender_original_label, author_gender_predicted_label)
            precision = precision_score(author_gender_original_label, author_gender_predicted_label)
            recall = recall_score(author_gender_original_label, author_gender_predicted_label)
            f1 = f1_score(author_gender_original_label, author_gender_predicted_label)

            acc_per_fold.append(accuracy)
            precision_per_fold.append(precision)
            recall_per_fold.append(recall)
            f1_score_per_fold.append(f1)

        sum_acc = 0
        sum_loss = 0
        sum_precision = 0
        sum_recall = 0
        sum_f1_score = 0

        for i, acc in enumerate(acc_per_fold):
            sum_acc = sum_acc + acc
            sum_loss = sum_loss+ loss_per_fold[i]
            sum_precision = sum_precision + precision_per_fold[i]
            sum_recall = sum_recall + recall_per_fold[i]
            sum_f1_score = sum_f1_score + f1_score_per_fold[i]

        avg_acc = sum_acc/10.0
        avg_precision = sum_precision /10.0
        avg_recall = sum_recall/10.0
        avg_f1 = sum_f1_score /10.0

        return acc_per_fold, precision_per_fold, recall_per_fold, f1_score_per_fold, avg_acc, avg_precision, avg_recall, avg_f1

In [26]:
cbow=1
cbow_trainable=1
sg=0 
sg_trainable=0 
data_split=2 
batch=32 
epochs=5

acc_per_fold, precision_per_fold, recall_per_fold, f1_score_per_fold, avg_acc, avg_precision, avg_recall, avg_f1 = ModelDLWithWordEmbedding(cbow=1, cbow_trainable=1, sg=0, sg_trainable=0, data_split=2, batch=32, epochs=5, model_name="GRU").train(posts, genders)
print("acc_per_fold: ", acc_per_fold)
print("precision_per_fold: ", precision_per_fold)
print("recall_per_fold: ", recall_per_fold)
print("f1_score_per_fold: ", f1_score_per_fold)
print("avg_acc: ", avg_acc)
print("avg_precision: ", avg_precision)
print("avg_recall: ", avg_recall)
print("avg_f1: ", avg_f1)

ValueError: Unrecognized keyword arguments passed to Embedding: {'input_length': 100, 'weights': [array([[-0.24736658,  0.07031919,  0.01407981, ..., -0.27251738,
         0.13224085,  0.0765908 ],
       [-0.12291483,  0.08461943,  0.06060226, ..., -0.17144129,
         0.17412944,  0.02834771],
       [-0.15936373,  0.06352039,  0.02524688, ..., -0.229536  ,
         0.12055779,  0.05851828],
       ...,
       [ 0.00078043,  0.02895817,  0.01444013, ..., -0.01905065,
         0.02032049,  0.02145052],
       [ 0.0083534 ,  0.05115167,  0.02525139, ..., -0.02803847,
         0.03329531,  0.03750573],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])]}